# Exploratory Data Analysis: Greenhouse Gases  
**Data Source**: Processed NOAA data (cleaned in `1_data_loading.ipynb`)  

## Key Notes  
- **Focus Gas**: CH₄ (Methane) - primary analysis target  
- **Data Quality**:  
  - Missing values: All rows kept (missing values → `NaN`)  
  - Negative values: Converted to `NaN` (non-physical concentrations)  
  - Data start dates: Gases have distinct collection start dates  
- Raw data pipeline documented in [`1_data_loading.ipynb`](../notebooks/1_data_loading.ipynb)  

## Data Files  
- **Input**: `../data/processed/all_ghg_aligned.csv`  
- **Output**: `../data/processed/all_ghg_aligned_nan.csv`  

# Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import linregress
from pathlib import Path
import os
from statsmodels.tsa.stattools import adfuller, kpss

# Make Directory

In [2]:
# automatic path handling
NOTEBOOK_PATH = Path.cwd() # gets current working directory
if 'notebooks' in str(NOTEBOOK_PATH): # if running from /notebooks/
    PROCESSED_DATA = NOTEBOOK_PATH.parent / 'data' / 'processed'
else: # if running from the repo root
    PROCESSED_DATA = NOTEBOOK_PATH / 'data' / 'processed'

# create folder (if necessary)
PROCESSED_DATA.mkdir(parents=True, exist_ok=True)

# Load Data

In [3]:
df_combined = pd.read_csv("../data/processed/all_ghg_aligned.csv", parse_dates=['date'])

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/all_ghg_aligned.csv'

In [ ]:
df_combined.head()

# EDA 

## Overview

In [ ]:
df_combined.describe()

## Datatypes

In [ ]:
df_combined.dtypes

## Null Values

In [ ]:
# There are two classes of null values to be concerned with:
# - null values at the beginning of a series are likely due to the different data collection start dates for each gas type
# - internal null values are null/missing values after the data collection start date for the particular gas.  These null values 
# likely need to be imputed.  

df_combined.isnull().sum()

In [ ]:
# the fill values for 'values' is -999.999.  This is essentially the same as a null value.  
# So, I will check on the number of fill values.  

fillvalue_counts = (df_combined == -999.999).sum()
fillvalue_counts

In [ ]:
# visualize location of null values for each feature

plt.figure(figsize=(6,8))
sns.heatmap(df_combined.isnull(), cmap='Paired', cbar=False, yticklabels=False)
plt.title('Missing Data Heatmap')
plt.show()

Null (NaN) values are data points that were not collected or recorded. Many null values occur at the beginning of the timeseries for each gas, except CO2, since measurement of the other gases began after the first CO2 measurement. 

measurement start dates:
- CO2: 1969-8-20
- CH4: 1983-5-6
- CO: 1989-7-7
- N2O: 1995-12-15
- SF6: 1995-12-15

I will not impute any null value that exist at dates earlier than the first measurement date for each gas.

## Negative Values

In [ ]:
# There are some negative values for gas concentration.  This doesn't make physical sense. 
# One possible explanation is that the GC sensor was zeroed incorrectly. Either way, I will 
# likely set them to NaN.  First, inspect:

neg_value_count = (df_combined.iloc[:,1:] < 0).sum()
print('----- Negative Value Count per Gas Type -----')
print(neg_value_count)

In [ ]:
# visualize the location of the negative values

plt.figure(figsize=(6,8))
sns.heatmap(df_combined.iloc[:,1:] < 0, cmap='Paired', cbar=False, yticklabels=False)
plt.title('Negative Value Heatmap')
plt.show()

In [ ]:
# replace negative values with NaN

df_combined.iloc[:,1:] = df_combined.iloc[:,1:].mask(df_combined.iloc[:,1:] < 0, np.nan)
                          
new_neg_count = (df_combined.iloc[:,1:] < 0).sum()
print('----- Remaininig Negative Values per Gas Type -----')
print(new_neg_count)

All NaN values that originate after the data collection start date will be imputed during preprocessing.  

## Store New Dateframe as CSV

In [ ]:
# save the preprocessed data to the processed data folder

print(f'Processed data will save to: {PROCESSED_DATA}')
df_combined.to_csv(PROCESSED_DATA / 'all_ghg_aligned_nan.csv', index=False)

## Analyze Long-Term Trends

In [ ]:
# plot the yearly average concentration values vs time to visualize the full trend of 
# atmospheric concentrations over time. 

# calculate yearly averages for each gas
df_combined['year'] = df_combined['date'].dt.year
yearly_avg_df = df_combined.groupby('year').mean(numeric_only=True)

unit_map = {
    'CH4' : 'ppb',
    'CO' : 'ppb',
    'CO2' : 'ppm',
    'H2' : 'ppb',
    'N2O' : 'ppb',
    'SF6' : 'ppt'
}

# plot
plt.figure(figsize=(12,6))
for gas in ['CH4','CO','CO2','H2','N2O','SF6']:
    label = f'{gas} ({unit_map[gas]})'
    plt.plot(yearly_avg_df.index, yearly_avg_df[gas], label=label, linewidth=2)

plt.title('Annual Mean Atmospheric GHG Concentrations', fontsize=18)
plt.ylabel('Concentration', fontsize=16)
plt.xlabel('Year', fontsize=16)
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
plt.legend(fontsize=16)
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Create a data table of the linear regression metrics

gas_trend_data = []

gases = {
    'CH4': {'unit': 'ppb', 'scale':1},
    'CO': {'unit': 'ppb', 'scale':1},
    'CO2': {'unit': 'ppm', 'scale':1},
    'H2': {'unit': 'ppb', 'scale':1},
    'N2O': {'unit': 'ppb', 'scale':1},
    'SF6': {'unit': 'ppt', 'scale':1}
}

for gas in gases.keys():
    valid_data = yearly_avg_df[gas].dropna()
    x = valid_data.index
    y = valid_data.values

    # linear regression
    slope, intercept, r_value, p_value, std_err = linregress(x,y)

    # calculate 95% confidence interval for the slope
    ci_low = slope - (1.96 * std_err)
    ci_high = slope + (1.96 * std_err)
    
    gas_trend_data.append({
        'Gas' : gas,
        'Slope' : f'{slope:.3f}',
        'Unit' : f'{gases[gas]['unit']}/yr',
        'R-sqrd' : f'{r_value**2:.3f}',
        'Std Err' : f'{std_err:.3f}',
        'p-val' : f'{p_value:.2e}',
        '95% CI Low' : f'{ci_low:.3f}',
        '95% CI high' : f'{ci_high:.3f}',
        'Start Yr' : x.min(),
        'End yr' : x.max()
    })

gas_trend_df = pd.DataFrame(gas_trend_data)
print(gas_trend_df.to_string(index=False))

## Visualize Full Data Series and Run Stationarity Tests

In [ ]:
# Run ADF and KPSS tests to determine if statistical properties, including mean, variance, and autocorrelation, 
# are stable (stationary) over time.  
# ADF - Augmented Dickey-Fuller test
# KPSS - Kwiatkowski-Phillips-Schmidt-Shin test

gas_columns = ['CH4','CO','CO2','H2','N2O','SF6']

# function to check stationarity
def check_stationarity(series, gas):
    adf_result = adfuller(series.dropna())
    kpss_result = kpss(series.dropna(), regression='c')

    print(f'ADF and KPSS tests for {gas}:')
    print(f'ADF statistic {adf_result[0]:.4f}')
    print(f'ADF p-value {adf_result[1]:.4f}')
    print(f'ADF critical values: {adf_result[4]}\n')
    
    print(f'KPSS statistic {kpss_result[0]:.4f}')
    print(f'KPSS p-value {kpss_result[1]:.4f}')
    print(f'KPSS critical values: {kpss_result[3]}\n')

    if adf_result[1] < 0.05 and kpss_result[1] > 0.05:
        print(f'the {gas} time series is likely stationary according to ADF and KPSS tests.\n')
    elif adf_result[1] > 0.05 and kpss_result[1] < 0.05:
        print(f'the {gas} time series is non-stationary according to ADF and KPSS tests.\n')
    elif adf_result[1] > 0.05 and kpss_result[1] > 0.05:
        print(f'the {gas} time series may be trend-stationary according to ADF and KPSS tests.\n')
    else: 
        print(f'the {gas} time series may be difference-stationary according to ADF and KPSS tests.\n')

# plot raw data and perform the ADF test
for gas in gas_columns:
    plt.figure(figsize=(10,4))
    plt.plot(df_combined[gas], label=gas)
    plt.title(f'{gas} Time Series', fontsize=16)
    plt.tight_layout()
    plt.show()

    check_stationarity(df_combined[gas], gas)

The ADF and KPSS values strongly indicate non-stationarity for CH4, CO2, N2O, and SF6, which isn't surprsing considering the substantial upward trajectory of the gas concentrations over time observed in the long-term trend analysis as well as what is well known about the accumulation of these gases in the atmosphere. Furthermore, all raw data plots suggest seasonality, which is also a known quality of GHG concentration data.  Seasonality will present autocorrelation, which also leads to non-stationarity.  These datasets will need to be differenced to remove the upward trend and to stabilize the mean for fitting a SARIMA model.  

All data series, with the possible exception of CO, have significant spikes, which may be outliers. STL (seasonal trend decomposition with LOESS) will be used to detect and handle outliers. 

## Data Frequency (per year)

In [ ]:
# visualize seasonal cycles in a snippet of the full dataset.  
# I will use [CH4] values between the years of 2013 - 2018 to avoid spikes.  

mask = (df_combined['date'] >= '2013-01-01') & (df_combined['date'] <= '2018-01-01')
ch4_subset = df_combined.loc[mask, 'CH4']

plt.figure(figsize=(12,4))
plt.plot(df_combined.loc[mask, 'date'], ch4_subset, linewidth=1.5)
plt.title('Atmospheric CH4 Concentration - Five Year Window', fontsize = 18)
plt.ylabel('Concentration (PPB)', fontsize=18)
plt.xlabel('Year', fontsize=18)
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)

plt.show()

The yearly seasonality is apparent in all gas dataset plots and highlighted here for a subset of CH4 data as an example.  The increasing trend can also be observed in this 5 year window.  

In [ ]:
# The seasonality appears as a single cycle per year for each gas. 
# Confirm the number of datapoints per year for each gas.

df_counts = df_combined.copy()

df_counts['year'] = df_counts['date'].dt.year # extract the year
yearly_counts = df_counts.groupby('year').count()
yearly_counts

In [ ]:
# As expected, there is variation in the number of data points per year.  
# I will determine and use the mode of each gas for signal decomposition, preprocessing, and modeling.

seasonal_mode = yearly_counts.replace(0, np.nan).mode().iloc[0] 
seasonal_mode

The yearly mode for each data series is 52, which means that the data is generally collected weekly.  